# Using AutoGen with motleycrew

Microsoft AutoGen is one of the most popular multi-agent frameworks. Motleycrew supports interaction with autogen in both directions: firstly, you can wrap an AutoGen chat as a motleycrew tool, which you can then give to any supported agent; secondly, you can give any motleycrew tool (which includes agents used as tools) to an AutoGen agent. 

The two options are described in detail below

Let's install external dependencies for the example.

In [1]:
%pip install duckduckgo-search==5.3.0b4

# install lunary if you need logging
%pip install lunary==1.1.5

  Using cached duckduckgo_search-5.3.0b4-py3-none-any.whl.metadata (18 kB)
Using cached duckduckgo_search-5.3.0b4-py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached lunary-1.1.5-py3-none-any.whl.metadata (1.6 kB)
Using cached lunary-1.1.5-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Integrating an AutoGen chat into motleycrew

If you want to use an AutoGen chat (say one from their many wonderful examples, or one you have already) as part of a motleycrew setup, you can wrap it as a tool, which you can then give to any other motleycrew-supported agent.

Let's create an AutoGen chat for automatic problem solving.
The code is taken from the example here: https://microsoft.github.io/autogen/docs/notebooks/agentchat_groupchat

In [3]:
%pip install pyautogen

Note: you may need to restart the kernel to use updated packages.


In [4]:
import autogen
import os

llm_config = {
    "config_list": [{"model": "gpt-4-turbo", "api_key": os.environ["OPENAI_API_KEY"]}],
    "cache_seed": None,
}

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/stefan/.config/sagemaker/config.yaml


In [5]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "examples/data/groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

If we were using plain AutoGen, we'd start the chat with something like this:
```
user_proxy.initiate_chat(
    manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
)
```

You see, the chat accepts an input and returns an output, just like a tool.  
Because of that, for using the chat in motleycrew, we can utilize the built-in AutoGenChatTool. Its prompt can either be a template or a plain string. Here we are creating a tool that searches arXiv for recent papers.

In [6]:
from motleycrew.tools.autogen_chat_tool import AutoGenChatTool

knowledge_retrieval_tool = AutoGenChatTool(
    name="retrieve_knowledge_by_topic",
    description="Search arxiv for the latest paper on a given topic "
                "and find its potential applications in software.",  # will be used in the prompt of the future agent that will use the tool
    prompt="Find a latest paper about {topic} on arxiv "
            "and find its potential applications in software.",  # this is the initial prompt for the AutoGen chat itself
    initiator=user_proxy,
    recipient=manager,
)

We can now give the tool to any agent and solve tasks with it.

In [7]:
from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent

crew = MotleyCrew()
writer = ReActToolCallingMotleyAgent(tools=[knowledge_retrieval_tool])

In [8]:
from motleycrew.tasks import SimpleTask

blog_post_task = SimpleTask(
        crew = crew,
        name="Produce blog post on the applications of latest advancements related to GPT-4",
        description="Using the insights provided by searching research papers, develop an engaging blog "
                    "post that highlights the most significant advancements on GPT-4 ant their applications.\n"
                    "Your post should be informative yet accessible, catering to a tech-savvy audience.\n"
                    "Make it sound cool, avoid complex words so it doesn't sound like AI. "
                    "Create a blog post of at least 4 paragraphs.",
        agent=writer,
    )

In [9]:
crew.run()

User_proxy (to chat_manager):

Find a latest paper about GPT-4 advancements on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------

Next speaker: User_proxy


>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: Coder

Coder (to chat_manager):

To fetch the latest paper about GPT-4 advancements from arXiv and explore its potential applications in software, the following python script uses the `arxiv` library that makes it easy to query the arXiv API, find the relevant papers about GPT-4, and display the title, summary, and categories of the latest paper. This will help us understand what kind of advancements have been made and the potential domains of application.

Here’s the python script that you will need to run:

```python
# filename: fetch_latest_gpt4_paper.py
import arxiv

# Search for papers 

[TaskUnit(status=done)]

In [10]:
from IPython.display import display, Markdown
display(Markdown(blog_post_task.output))

Final Answer: 

**Unveiling the Magic of GPT-4: Transforming Language and Culture**

In the ever-evolving world of artificial intelligence, GPT-4 stands out as a beacon of innovation, pushing the boundaries of what's possible in language processing. Imagine a world where language barriers crumble, and cultural nuances are effortlessly understood. That's the promise of GPT-4, especially when it comes to translating complex idiomatic expressions in East Asian languages. This isn't just about words; it's about capturing the essence of culture and context, making communication more authentic and relatable.

One of the coolest applications of GPT-4 is in the realm of multilingual customer support chatbots. These aren't your average bots; they're culturally savvy, understanding and employing idiomatic language to create natural interactions. Imagine chatting with a bot that gets your cultural references and responds in a way that feels genuinely human. It's like having a conversation with a local, no matter where you are in the world. This level of cultural sensitivity can revolutionize customer engagement, making interactions more meaningful and effective.

For authors and creators, GPT-4 opens up a world of possibilities in translation tools. Picture a tool that not only translates your work but also preserves the cultural meanings and sentiments behind it. Whether you're an author looking to share your story with a global audience or a writer wanting to incorporate foreign idioms authentically, GPT-4 has got you covered. It's like having a cultural ambassador by your side, ensuring your message resonates with readers from different backgrounds.

The media industry, too, can benefit from GPT-4's advancements. Automated contextual subtitling is a game-changer, generating subtitles that go beyond direct translation to ensure cultural relevance and comprehension. Imagine watching a foreign film where the subtitles capture not just the dialogue but the cultural nuances, enhancing your viewing experience. It's like having a personal translator who understands the subtleties of language and culture, making global content more accessible and enjoyable.

In the world of education, GPT-4 is a powerful ally for language learners. With contextual learning modules, students can dive into the world of idioms, understanding their use in everyday conversation. Interactive scenarios bring these expressions to life, helping learners grasp their meaning and usage in different contexts. It's like having a language coach who not only teaches you the words but also the stories behind them, enriching your learning journey.

GPT-4 is not just about language; it's about bridging cultures and enhancing communication. Whether it's through chatbots, translation tools, media applications, or educational platforms, the potential is limitless. As we continue to explore and harness the power of GPT-4, one thing is clear: the future of language processing is here, and it's more exciting than ever.

The writer agent used the AutoGen chat tool to retrieve data for the post!



## Using any motleycrew-supported tools with Autogen agents

Now let's do it the other way around: give tools (or agents, for that matter) from motleycrew to AutoGen agents. The code is based on https://microsoft.github.io/autogen/docs/tutorial/tool-use

In [11]:
from motleycrew.tools import MotleyTool, RetryConfig
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = MotleyTool.from_supported_tool(
    DuckDuckGoSearchRun(),
    retry_config=RetryConfig(max_retries=5)  # for retrying rate limit errors
)  # Any tools or even motleycrew's agents can be converted to MotleyTool like this!

# Let's first define the assistant agent that suggests tool calls.
assistant = autogen.ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can provide useful and up-to-date data using web search. "
    "Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = autogen.ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Make an autogen-compatible tool from the MotleyTool.
autogen_tool = search_tool.to_autogen_tool()

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="search_tool", description="Web search tool")(autogen_tool)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="search_tool")(autogen_tool)

<function motleycrew.tools.tool.MotleyTool.to_autogen_tool.<locals>.autogen_tool_fn(input: str) -> str>

In [12]:
chat_result = user_proxy.initiate_chat(assistant, message="What was the first computer?")

User (to Assistant):

What was the first computer?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_lp7DaZO1WUlu7YsMZazwsZX2): search_tool *****
Arguments: 
{"input":"first computer"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_tool...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_lp7DaZO1WUlu7YsMZazwsZX2) *****
Learn how the English mathematician and inventor Charles Babbage designed the Difference Engine and the Analytical Engine, the first mechanical digital computers. Explore their features, components, and challenges in this article from Britannica. ENIAC was built by the United States during World War II and completed in 1946. It was the first general-purpose digital computer that

In [17]:
final_result = 'The agent used the search tool!'
print(final_result)

The agent used the search tool!
